In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os

os.listdir("/content/drive/MyDrive/archive")

['online_retail_II.csv']

In [6]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/archive/online_retail_II.csv")
print(df.shape)
df.head()

(1067371, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
import os
import pandas as pd

folder = "/content/drive/MyDrive/archive"
files = os.listdir(folder)

for f in files:
    if f.endswith(".xlsx"):
        df = pd.read_excel(os.path.join(folder, f))
    elif f.endswith(".csv"):
        df = pd.read_csv(os.path.join(folder, f))

print(df.shape)
df.head()

(1067371, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [9]:
df.isnull().sum()

,0
Invoice,0
StockCode,0
Description,4382
Quantity,0
InvoiceDate,0
Price,0
Customer ID,243007
Country,0


In [10]:
df.duplicated().sum()

np.int64(34335)

In [14]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [16]:
df.columns = df.columns.str.strip().str.replace(" ", "").str.lower()

In [17]:
df.columns.tolist()

['invoice',
 'stockcode',
 'description',
 'quantity',
 'invoicedate',
 'price',
 'customerid',
 'country']

In [18]:
[col for col in df.columns if 'price' in col or 'cost' in col or 'amount' in col]

['price']

In [19]:
PRICE_COL = 'price'
QTY_COL = 'quantity'
CUST_COL = 'customerid'

In [20]:
validation_report = {
    "total_rows": len(df),
    "missing_customerid": df[CUST_COL].isnull().sum() if CUST_COL in df.columns else "N/A",
    "duplicate_rows": df.duplicated().sum(),
    "invalid_quantity": (df[QTY_COL] <= 0).sum() if QTY_COL in df.columns else "N/A",
    "invalid_price": (df[PRICE_COL] <= 0).sum() if PRICE_COL in df.columns else "N/A"
}

validation_report

{'total_rows': 1067371,
 'missing_customerid': np.int64(243007),
 'duplicate_rows': np.int64(34335),
 'invalid_quantity': np.int64(22950),
 'invalid_price': np.int64(6207)}

In [21]:
QTY_COL = 'quantity'
CUST_COL = 'customerid'

# detect price column safely
PRICE_COL = None
for col in df.columns:
    if 'price' in col or 'cost' in col or 'amount' in col:
        PRICE_COL = col

PRICE_COL

'price'

In [22]:
before = len(df)
df = df.drop_duplicates()
after = len(df)

print(f"Duplicates removed: {before - after}")

Duplicates removed: 34335


In [23]:
before = len(df)
df = df.dropna(subset=[CUST_COL])
after = len(df)

print(f"Rows removed (missing customer): {before - after}")

Rows removed (missing customer): 235151


In [24]:
if PRICE_COL:
    before = len(df)
    df = df[df[PRICE_COL] > 0]
    after = len(df)
    print(f"Rows removed (invalid price): {before - after}")
else:
    print("No price column found – skipped price validation")

Rows removed (invalid price): 70


In [25]:
df['invoicedate'] = pd.to_datetime(df['invoicedate'], errors='coerce')
df = df.dropna(subset=['invoicedate'])

In [26]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 797815 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      797815 non-null  object        
 1   stockcode    797815 non-null  object        
 2   description  797815 non-null  object        
 3   quantity     797815 non-null  int64         
 4   invoicedate  797815 non-null  datetime64[ns]
 5   price        797815 non-null  float64       
 6   customerid   797815 non-null  float64       
 7   country      797815 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 54.8+ MB


,invoice,stockcode,description,quantity,invoicedate,price,customerid,country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [28]:
df.columns = df.columns.str.strip().str.replace(" ", "").str.lower()

In [30]:
print(df.columns.tolist())

['invoice', 'stockcode', 'description', 'quantity', 'invoicedate', 'price', 'customerid', 'country']


In [31]:
PRICE_COL = 'price'
QTY_COL = 'quantity'
CUST_COL = 'customerid'
DATE_COL = 'invoicedate'

In [32]:
def validate_data(df):
    return {
        "total_rows": len(df),
        "missing_customerid": df[CUST_COL].isnull().sum(),
        "duplicate_rows": df.duplicated().sum(),
        "invalid_quantity": (df[QTY_COL] <= 0).sum(),
        "invalid_price": (df[PRICE_COL] <= 0).sum()
    }

validate_data(df)

{'total_rows': 797815,
 'missing_customerid': np.int64(0),
 'duplicate_rows': np.int64(0),
 'invalid_quantity': np.int64(18390),
 'invalid_price': np.int64(0)}

In [34]:
before = len(df)

df = df.dropna(subset=[CUST_COL])
df = df[df[QTY_COL] > 0]
df = df[df[PRICE_COL] > 0]

after = len(df)

print(f"Rows removed during cleaning: {before - after}")
print("Clean rows:", after)

Rows removed during cleaning: 18390
Clean rows: 779425


In [35]:
df['revenue'] = df[QTY_COL] * df[PRICE_COL]
df[['quantity', 'price', 'revenue']].head()

,quantity,price,revenue
0,12,6.95,83.4
1,12,6.75,81.0
2,12,6.75,81.0
3,48,2.10,100.8
4,24,1.25,30.0


In [36]:
daily_report = (
    df.groupby(df[DATE_COL].dt.date)
      .agg(
          total_orders=('invoice', 'nunique'),
          total_revenue=('revenue', 'sum'),
          total_customers=('customerid', 'nunique')
      )
      .reset_index()
)

daily_report.head()

,invoicedate,total_orders,total_revenue,total_customers
0,2009-12-01,98,43894.87,91
1,2009-12-02,110,52762.06,94
2,2009-12-03,122,67413.62,106
3,2009-12-04,80,33913.81,76
4,2009-12-05,30,9803.05,26


In [37]:
daily_report.to_csv("daily_operations_report.csv", index=False)

In [38]:
invalid_quantity_df = df[df['quantity'] <= 0]

invalid_quantity_df.shape

(0, 9)

In [39]:
invalid_quantity_df.to_csv("invalid_quantity_log.csv", index=False)

In [40]:
clean_df = df[df['quantity'] > 0].copy()

clean_df.shape

(779425, 9)

In [41]:
clean_df['revenue'] = clean_df['quantity'] * clean_df['price']

kpi_summary = {
    "total_orders": clean_df['invoice'].nunique(),
    "total_customers": clean_df['customerid'].nunique(),
    "total_revenue": round(clean_df['revenue'].sum(), 2)
}

kpi_summary

{'total_orders': 36969,
 'total_customers': 5878,
 'total_revenue': np.float64(17374804.27)}